In [ ]:
import cobra
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



models = {
    'EC model': cobra.io.read_sbml_model(""),
    'Original model': cobra.io.read_sbml_model(""),
}

for model in models.values():
    model.objective = "BIOMASS_Cl_DSM_WT_46p666M1"



co_uptake_range = np.arange(0, 51, 1)

results_growth = []
results_acetate = []
results_ethanol = []
results_co2 = []
results_enzyme_pool = []


experimental_co_uptakes = [35.878, 38.5]
experimental_growth_rates = [0.058, 0.060]
experimental_acetate_mean = [0.292, 2.052]
experimental_acetate_sd = [0.119, 0]
experimental_ethanol_mean = [4.1, 3.948]
experimental_ethanol_sd = [0.756, 0]
experimental_co2_mean = [21.769, 23]
experimental_co2_sd = [1.15, 0]


for model_name, model in models.items():
    model.reactions.get_by_id('EX_fru_e').bounds = (0, 0)  
    model.reactions.get_by_id('BIOMASS_Cl_DSM_WT_46p666M1').bounds = (0, 1000) 
    
    for co_uptake in co_uptake_range:
        model.reactions.get_by_id('EX_co_e').bounds = (-co_uptake, -co_uptake)
        
        solution = model.optimize()


        if solution.status == 'optimal':
            result_growth = {'Model': model_name, 'CO Uptake': co_uptake, 'Growth Rate': solution.objective_value}
            results_growth.append(result_growth)


            result_acetate = {'Model': model_name, 'CO Uptake': co_uptake}
            result_ethanol = {'Model': model_name, 'CO Uptake': co_uptake}
            result_co2 = {'Model': model_name, 'CO Uptake': co_uptake}

            if 'EX_ac_e' in model.reactions:
                acetate_flux = solution.fluxes['EX_ac_e']
                result_acetate['Acetate Production'] = acetate_flux
            if 'EX_etoh_e' in model.reactions:
                ethanol_flux = solution.fluxes['EX_etoh_e']
                result_ethanol['Ethanol Production'] = ethanol_flux
            if 'EX_co2_e' in model.reactions:
                co2_flux = solution.fluxes['EX_co2_e']
                result_co2['CO2 Emission'] = co2_flux

            results_acetate.append(result_acetate)
            results_ethanol.append(result_ethanol)
            results_co2.append(result_co2)

results_growth_df = pd.DataFrame(results_growth)
results_acetate_df = pd.DataFrame(results_acetate)
results_ethanol_df = pd.DataFrame(results_ethanol)
results_co2_df = pd.DataFrame(results_co2)



plt.figure(figsize=(10, 6))
plt.scatter(experimental_co_uptakes, experimental_growth_rates, color='blue', label='Experimental Growth')
for model_name, model_data in results_growth_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['CO Uptake'], model_data['Growth Rate'], linestyle=linestyle, color='blue', label=f'{model_name} (Growth Rate)')
plt.xlabel('CO Uptake Rate [mmol/gCDW/h]', fontsize=14)  
plt.ylabel('Growth Rate [h^-1]', fontsize=14) 
plt.title('Model Performance: Growth Rate', fontsize=16) 
plt.legend(fontsize=14) 
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12) 
plt.grid(True)
plt.savefig("growth_rate.png", dpi=300)
plt.show()


plt.figure(figsize=(10, 6))
plt.errorbar(experimental_co_uptakes, experimental_acetate_mean, yerr=experimental_acetate_sd, fmt='o', color='orange', label='Experimental Acetate')
plt.errorbar(experimental_co_uptakes, experimental_ethanol_mean, yerr=experimental_ethanol_sd, fmt='o', color='green', label='Experimental Ethanol')
for model_name, model_data in results_acetate_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['CO Uptake'], model_data['Acetate Production'], linestyle=linestyle, color='orange', label=f'{model_name} (Acetate Production)')
for model_name, model_data in results_ethanol_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['CO Uptake'], model_data['Ethanol Production'], linestyle=linestyle, color='green', label=f'{model_name} (Ethanol Production)')
plt.xlabel('CO Uptake Rate [mmol/gCDW/h]', fontsize=14)  
plt.ylabel('Production Rate [mmol/gCDW/h]', fontsize=14) 
plt.title('Model Performance: Acetate and Ethanol Production', fontsize=16)  
plt.legend(fontsize=14) 
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12) 
plt.grid(True)
plt.savefig("products.png", dpi=300)
plt.show()


plt.figure(figsize=(10, 6))
plt.errorbar(experimental_co_uptakes, experimental_co2_mean, yerr=experimental_co2_sd, fmt='o', color='orangered', label='Experimental CO2')
for model_name, model_data in results_co2_df.groupby('Model'):
    plt.plot(model_data['CO Uptake'], model_data['CO2 Emission'], label=f'{model_name} (CO2 Emission)')
plt.xlabel('CO Uptake Rate [mmol/gCDW/h]', fontsize=14)  
plt.ylabel('CO2 Emission Rate [mmol/gCDW/h]', fontsize=14) 
plt.title('Model Performance: CO2 Emission', fontsize=16)  
plt.legend(fontsize=14)  
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.grid(True)
plt.savefig("co2_emission.png", dpi=300)
plt.show()

In [ ]:
import cobra
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


models = {
    'EC model': cobra.io.read_sbml_model("/"),
    'Original model': cobra.io.read_sbml_model("/"),
}

for model in models.values():
    model.objective = "BIOMASS_Cl_DSM_WT_46p666M1"


h2_uptake_range = np.arange(25, 81, 1)

results_growth = []
results_acetate = []
results_ethanol = []


for model_name, model in models.items():

    for h2_uptake in h2_uptake_range:
        model.reactions.get_by_id('EX_h2_e').bounds = (-h2_uptake, -h2_uptake)  # Set the lower and upper bounds for CO uptake
        model.reactions.get_by_id('EX_fru_e').bounds = (-0.615, -0.615)  # Set the lower and upper bounds for fructose uptake
        model.reactions.get_by_id('EX_co2_e').bounds = (-100, -15.861)


        solution = model.optimize()

  
        result_growth = {'Model': model_name, 'H2 Uptake': h2_uptake, 'Growth Rate': solution.objective_value}
        results_growth.append(result_growth)

     
        result_acetate = {'Model': model_name, 'H2 Uptake': h2_uptake}
        result_ethanol = {'Model': model_name, 'H2 Uptake': h2_uptake}
        
        if 'EX_ac_e' in model.reactions:
            acetate_flux = solution.fluxes['EX_ac_e']
            result_acetate['Acetate Production'] = acetate_flux
        if 'EX_etoh_e' in model.reactions:
            ethanol_flux = solution.fluxes['EX_etoh_e']
            result_ethanol['Ethanol Production'] = ethanol_flux

        results_acetate.append(result_acetate)
        results_ethanol.append(result_ethanol)


results_growth_df = pd.DataFrame(results_growth)
results_acetate_df = pd.DataFrame(results_acetate)
results_ethanol_df = pd.DataFrame(results_ethanol)


plt.figure(figsize=(10, 6))
for model_name, model_data in results_growth_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['H2 Uptake'], model_data['Growth Rate'], linestyle=linestyle, color='blue', label=f'{model_name} (Growth Rate)')
plt.xlabel('H2 Uptake Rate [mmol/gCDW/h]',fontsize=14)
plt.ylabel('Growth Rate [h^-1]',fontsize=14)
plt.title('Model Performance: Growth Rate',fontsize=16)
plt.legend(fontsize=14)
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.grid(True)
plt.savefig("growth_rate.png", dpi=300)
plt.show()


plt.figure(figsize=(10, 6))
for model_name, model_data in results_acetate_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['H2 Uptake'], model_data['Acetate Production'], linestyle=linestyle, color='orange', label=f'{model_name} (Acetate Production)')
for model_name, model_data in results_ethanol_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['H2 Uptake'], model_data['Ethanol Production'], linestyle=linestyle, color='green', label=f'{model_name} (Ethanol Production)')
plt.xlabel('H2 Uptake Rate [mmol/gCDW/h]',fontsize=14)
plt.ylabel('Production Rate [mmol/gCDW/h]',fontsize=14)
plt.title('Model Performance: Acetate and Ethanol Production',fontsize=16)
plt.legend(fontsize=14)
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.grid(True)
plt.savefig("products.png", dpi=300)
plt.show()



In [ ]:
import cobra
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


models = {
    'EC model': cobra.io.read_sbml_model(""),
    'Original model': cobra.io.read_sbml_model(""),
}

for model_name, model in models.items():
    model.reactions.get_by_id('EX_co2_e').bounds = (-1000, 1000)

for model in models.values():
    model.objective = "BIOMASS_Cl_DSM_WT_46p666M1"


co_uptake_range = np.arange(0, 51, 1)

results_growth = []
results_acetate = []
results_ethanol = []
results_co2 = []
results_enzyme_pool = []


experimental_co_uptakes = [40.132]
experimental_growth_rates = [0.043]
experimental_acetate_mean = [2.086]
experimental_acetate_sd = [0.119]
experimental_ethanol_mean = [4.784]
experimental_ethanol_sd = [0.756]
experimental_co2_mean = [25.659]
experimental_co2_sd = [1.15]
experimental_co_uptakes.append(26.09)
experimental_growth_rates.append(0.050)
experimental_acetate_mean.append(2.308)
experimental_ethanol_mean.append(2.611)
experimental_co2_mean.append(9.838)

for model_name, model in models.items():
    model.reactions.get_by_id('EX_co2_e').lower_bound = 0

    for co_uptake in co_uptake_range:
        model.reactions.get_by_id('EX_co_e').bounds = (-co_uptake, -co_uptake)  # Set the lower and upper bounds for CO uptake
        model.reactions.get_by_id('EX_fru_e').bounds = (0, 0)  # Set the lower and upper bounds for fructose uptake
        model.reactions.get_by_id('EX_h2_e').bounds = (-1000, 0)  # Set the lower and upper bounds for fructose uptake
        

  
        solution = model.optimize()

       
        if solution.status == 'optimal':
            result_growth = {'Model': model_name, 'CO Uptake': co_uptake, 'Growth Rate': solution.objective_value}
            results_growth.append(result_growth)

            
            result_acetate = {'Model': model_name, 'CO Uptake': co_uptake}
            result_ethanol = {'Model': model_name, 'CO Uptake': co_uptake}
            result_co2 = {'Model': model_name, 'CO Uptake': co_uptake}

            if 'EX_ac_e' in model.reactions:
                acetate_flux = solution.fluxes['EX_ac_e']
                result_acetate['Acetate Production'] = acetate_flux
            if 'EX_etoh_e' in model.reactions:
                ethanol_flux = solution.fluxes['EX_etoh_e']
                result_ethanol['Ethanol Production'] = ethanol_flux
            if 'EX_co2_e' in model.reactions:
                co2_flux = solution.fluxes['EX_co2_e']
                result_co2['CO2 Emission'] = co2_flux

            results_acetate.append(result_acetate)
            results_ethanol.append(result_ethanol)
            results_co2.append(result_co2)


results_growth_df = pd.DataFrame(results_growth)
results_acetate_df = pd.DataFrame(results_acetate)
results_ethanol_df = pd.DataFrame(results_ethanol)
results_co2_df = pd.DataFrame(results_co2)


plt.figure(figsize=(10, 6))
plt.scatter(experimental_co_uptakes, experimental_growth_rates, color='blue', label='Experimental Growth')
for model_name, model_data in results_growth_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['CO Uptake'], model_data['Growth Rate'], linestyle=linestyle, color='blue', label=f'{model_name} (Growth Rate)')
plt.xlabel('CO Uptake Rate [mmol/gCDW/h]', fontsize=14)
plt.ylabel('Growth Rate [h^-1]',fontsize=14)
plt.title('Model Performance: Growth Rate',fontsize=16)
plt.legend(fontsize=14)
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.grid(True)
plt.savefig("growth_rate.png", dpi=300)
plt.show()


plt.figure(figsize=(10, 6))
plt.errorbar(experimental_co_uptakes, experimental_acetate_mean, yerr=experimental_acetate_sd, fmt='o', color='orange', label='Experimental Acetate')
plt.errorbar(experimental_co_uptakes, experimental_ethanol_mean, yerr=experimental_ethanol_sd, fmt='o', color='green', label='Experimental Ethanol')
for model_name, model_data in results_acetate_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['CO Uptake'], model_data['Acetate Production'], linestyle=linestyle, color='orange', label=f'{model_name} (Acetate Production)')
for model_name, model_data in results_ethanol_df.groupby('Model'):
    linestyle = '--' if model_name == 'Original model' else '-'
    plt.plot(model_data['CO Uptake'], model_data['Ethanol Production'], linestyle=linestyle, color='green', label=f'{model_name} (Ethanol Production)')
plt.xlabel('CO Uptake Rate [mmol/gCDW/h]',fontsize=14)
plt.ylabel('Production Rate [mmol/gCDW/h]',fontsize=14)
plt.title('Model Performance: Acetate and Ethanol Production',fontsize=16)
plt.legend(fontsize=14)
plt.xticks(fontsize=12)  
plt.yticks(fontsize=12)  
plt.grid(True)
plt.savefig("products.png", dpi=300)
plt.show()




plt.figure(figsize=(10, 6))
plt.errorbar(experimental_co_uptakes, experimental_co2_mean, yerr=experimental_co2_sd, fmt='o', color='orangered', label='Experimental CO2')
for model_name, model_data in results_co2_df.groupby('Model'):
    plt.plot(model_data['CO Uptake'], model_data['CO2 Emission'], label=f'{model_name} (CO2 Emission)')
plt.xlabel('CO Uptake Rate [mmol/gCDW/h]',fontsize=14)
plt.ylabel('CO2 Emission Rate [mmol/gCDW/h]',fontsize=14)
plt.title('Model Performance: CO2 Emission',fontsize=16)
plt.legend(fontsize=14)
plt.xticks(fontsize=12) 
plt.yticks(fontsize=12) 
plt.grid(True)
plt.savefig("co2_emission.png", dpi=300)
plt.show()
